In [1]:
import torch
from torch.nn import functional as F
from torch import optim
from torch import nn
import sys
sys.path.insert(0, '../')

from config import Config, ModelType, PreTrainedType
from transformers import BertModel, BertConfig, BertForSequenceClassification
from models import load_model
from dataset import REDataset, split_train_test_loader
from utils import set_seed

In [2]:
model = load_model(model_type=ModelType.VanillaBert)

Load Model...	asdfasdfs
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

In [2]:
LOAD = "../saved_models/BertModel_bert-base-multilingual-cased_20210417144420/BertModel_bert-base-multilingual-cased_ep(00)acc(0.5004)loss(0.0108)id(20210417144420).pth"

In [10]:
bert_config = BertConfig.from_pretrained(PreTrainedType.BertMultiLingual)
bert_config.num_labels = 42
# bert_config.output_hidden_states = True
model = BertForSequenceClassification.from_pretrained(PreTrainedType.BertMultiLingual, config=bert_config)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [11]:
set_seed()
dataset = REDataset(device='cpu')
train_loader, valid_loader = split_train_test_loader(dataset, test_size=0.25)

Seed set as 42
Load Tokenizer...	done!
Load raw data...	done!
Apply Tokenization...	done!


In [12]:
for sents, labels in valid_loader:
    break

In [18]:
hidden = outputs.last_hidden_state

In [22]:
IDX = 0
hidden[:, IDX, :].size()

torch.Size([512, 768])

In [45]:
class temp(nn.Module):
    def __init__(self, model=model):
        super(temp, self).__init__()
        self.bert = model.bert

    def forward(self, input_ids, token_type_ids, attention_mask):
        return self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

In [49]:
vanilla = temp(model=model)
optimizer = optim.Adam(vanilla.parameters(), lr=0.5)
criterion = nn.CrossEntropyLoss()

In [33]:
for i in bert.modules():
    break

In [50]:
output = vanilla(**sents)

In [56]:
output.last_hidden_state.size()

torch.Size([512, 128, 768])

In [98]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [18]:
class VanillaBert(nn.Module):
    def __init__(self, model_type:str=ModelType.SequenceClf, pretrained_type: str=PreTrainedType.BertMultiLingual, num_labels: int=Config.NumClasses, pooler_idx: int='cls', load_state_dict=None):
        super(VanillaBert, self).__init__()
        print(model_type)
        # idx: index of hidden state to extract from output hidden states. It is CLS hidden states for index 0.
        self.idx = 0 if pooler_idx in ['cls', 0] else pooler_idx 
        self.backbone = self.load_bert(model_type=model_type, pretrained_type=pretrained_type, num_labels=num_labels)
        self.layernorm = nn.LayerNorm(768) # 768: output length of BERT, or backbone
        self.dropout = nn.Dropout()
        self.relu = nn.ReLU()
        self.linear = nn.Linear(in_features=768, out_features=num_labels)

    def forward(self, input_ids, token_type_ids, attention_mask):
        x = self.backbone(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        x = x.output_hidden_state[:, self.idx, :]
        x = self.layernorm(x)
        x = self.dropout(x)
        x = self.relu(x)
        output = self.linear(x)
        return output

    @staticmethod
    def load_bert(model_type, pretrained_type, num_labels):
        bert_config = BertConfig.from_pretrained(pretrained_type)
        bert_config.num_labels = num_labels

        if model_type == ModelType.SequenceClf:    
            model = BertForSequenceClassification.from_pretrained(pretrained_type, config=bert_config)
            model = model.bert
            
        elif model_type == ModelType.Base:
            raise NotImplementedError()

        return model

In [19]:
model = VanillaBert()

BertForSequenceClassification
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not

In [20]:
model.backbone

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [84]:
temp = vanilla.bert.encoder.layer[0].intermediate.dense.parameters()
par = list(temp)
par[0].grad

In [86]:
par = list(temp)

In [90]:
par[0].grad

In [63]:
loss = criterion(output.pooler_output, labels)
loss

tensor(6.5733, grad_fn=<NllLossBackward>)

In [92]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [93]:
temp = vanilla.bert.encoder.layer[0].intermediate.dense.parameters()
par = list(temp)
par[0].grad

tensor([[-1.3005e-05, -2.9100e-06,  1.3296e-05,  ..., -1.7892e-06,
         -3.8926e-06, -1.2175e-05],
        [-1.1622e-05,  1.5063e-06,  3.8757e-05,  ..., -9.9145e-05,
         -2.8575e-05,  4.5431e-05],
        [ 3.4346e-05,  3.4284e-05,  3.3967e-05,  ..., -1.9772e-05,
         -3.3613e-06,  5.6124e-06],
        ...,
        [-4.9324e-06,  1.1256e-04, -6.2499e-05,  ..., -4.2833e-05,
          1.2524e-05,  4.7053e-05],
        [ 4.7357e-05, -9.0478e-06, -2.4913e-05,  ...,  7.1708e-06,
          9.2066e-06, -3.8083e-05],
        [ 1.9760e-06,  1.0706e-05,  1.1165e-04,  ..., -7.2068e-05,
          4.8590e-05,  1.5547e-05]])

In [11]:
outputs = model(**sents)

In [17]:
preds = F.softmax(outputs.pooler_output, dim=1)

In [19]:
preds[0]

tensor([0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080,
        0.0080, 0.0080, 0.0079, 0.0070, 0.0011, 0.0078, 0.0080, 0.0012, 0.0080,
        0.0011, 0.0011, 0.0080, 0.0080, 0.0080, 0.0079, 0.0080, 0.0077, 0.0011,
        0.0080, 0.0011, 0.0011, 0.0011, 0.0011, 0.0012, 0.0079, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0012, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 

In [ ]:
model_type: str = ModelType.SequenceClf,
pretrained_type: str = PreTrainedType.BertMultiLingual,
num_classes: int = Config.NumClasses,
load_state_dict: str = None,

In [ ]:
load_model()